### CATEGORIZACIÓN Y ANÁLISIS DE SENTIMIENTOS DE ARTÍCULOS DE NOTICIAS
#### Preparación de los artículos procesados
#### Entrenamiento y prueba del modelo de Support Vector Machine - SVM

In [1]:
# Se importan las librerías necesarias para el preprocesamiento de los datos.
import pandas as pd
from scipy import sparse

# Se importan los scripts necesarios para preparamos los datos y entrenar el modelo.
from topic_modeling_prep import TopicModelingPrep
from svm_news_classifier import SVMTopicClassifier, main

In [ ]:
# Se cargan los datos procesados de noticias en un DataFrame.
processed_news_df = pd.read_csv("./data/processed_news_dataset_sample.csv", encoding='utf-8-sig',na_values=['', 'NA', 'null', 'NULL', 'NaN'])
processed_news_df.info()

In [ ]:
processed_news_df = processed_news_df.dropna(subset=['processed_text'])
processed_news_df.info()

In [ ]:
# Se preparan los datos para el entrenamiento del modelo.
prep = TopicModelingPrep()
topic_df, artifacts = prep.prepare_data(processed_news_df)

In [ ]:
# Se muestra el dataframe con los datos preparados.
display(topic_df)

In [ ]:
# Se muestra el contenido de los artefactos generados.
display(artifacts)

In [ ]:
# Se entrena y se prube el modelo de clasificación de noticias.
classifier, topic_distributions, evaluation_results = main(topic_df, artifacts)

# Se imprime el reporte de clasificación detallado.
print("\nDetailed Classification Report:")
print(pd.DataFrame(evaluation_results['classification_report']).T)

In [ ]:
parameter_combinations = [
    # Balanced configuration
    {
        'n_topics': 20,
        'learning_offset': 25.0,
        'kernel': 'rbf',
        'C': 1.0,
        'gamma': 'scale',
        'class_balance': 'balanced',
        'decision_function_shape': 'ovr'
    },
    # Linear kernel configuration
    {
        'n_topics': 25,
        'learning_offset': 15.0,
        'kernel': 'linear',
        'C': 0.1,
        'gamma': 'auto',
        'class_balance': 'balanced',
        'decision_function_shape': 'ovo'
    },
    # Polynomial kernel configuration
    {
        'n_topics': 30,
        'learning_offset': 35.0,
        'kernel': 'poly',
        'C': 10.0,
        'gamma': 'scale',
        'class_balance': 'balanced',
        'decision_function_shape': 'ovr'
    }
]

results = []
for params in parameter_combinations:
    print(f"\nTesting parameters: {params}")
    
    classifier = SVMTopicClassifier(**params)
    topic_distributions, top_words = classifier.fit_transform_lda(topic_df, artifacts)
    evaluation_results = classifier.train_classifier(topic_df, topic_distributions)
    
    results.append({
        'parameters': params,
        'accuracy': evaluation_results['classification_report']['accuracy'],
        'weighted_f1': evaluation_results['classification_report']['weighted avg']['f1-score']
    })

In [ ]:
# Print results
for result in results:
    print("\nParameters:", result['parameters'])
    print(f"Accuracy: {result['accuracy']:.3f}")
    print(f"Weighted F1: {result['weighted_f1']:.3f}")

# Find best combination
best_result = max(results, key=lambda x: x['weighted_f1'])
print("\nBest combination:")
print(f"Parameters: {best_result['parameters']}")
print(f"Weighted F1: {best_result['weighted_f1']:.3f}")